<a href="https://colab.research.google.com/github/mlichter2/cesium/blob/master/colab_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install rasterio
!pip install geopandas

     |████████████████████████████████| 18.2MB 228kB/s 
     |████████████████████████████████| 972kB 9.0MB/s 
     |████████████████████████████████| 10.9MB 30.2MB/s 
     |████████████████████████████████| 14.7MB 251kB/s 


In [4]:

import tensorflow as tf

import numpy as np
import imageio
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_input_vgg16
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_input_resnet50
from tensorflow.keras.losses import binary_crossentropy, mean_squared_error, kullback_leibler_divergence, \
    mean_absolute_error, cosine_similarity
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, CSVLogger
from skimage.transform import resize
from tensorflow.keras.initializers import RandomNormal, GlorotNormal, Zeros
import cv2
import pandas as pd
import os
import json
import rasterio
import geopandas as gpd
import datetime
from scipy.ndimage import gaussian_filter1d

IMAGE_DIM_X = 100
IMAGE_DIM_Y = 300



In [ ]:


# def custom_loss(y_true, y_pred):
#
#   bce = binary_crossentropy(y_true[:, :-1], y_pred[:, :-1])
#   bce2 = binary_crossentropy(y_true[:, -1], y_pred[:, -1])
#   return 2 * bce * y_true[:, -1] + 0.5 * bce2

def custom_loss_bce(y_true, y_pred):
    # bce = binary_crossentropy(y_true[:, :-1], y_pred[:, :-1])
    l1 = binary_crossentropy(y_true[:, :25], y_pred[:, :25])
    l2 = binary_crossentropy(y_true[:,75 :100], y_pred[:,75 :100])
    l3 = binary_crossentropy(y_true[:, 100:125], y_pred[:, 100:125])
    l4 = binary_crossentropy(y_true[:, 175:200], y_pred[:, 175:200])
    l5 = binary_crossentropy(y_true[:, 25:75], y_pred[:, 25:75])
    l6 = binary_crossentropy(y_true[:, 125:175], y_pred[:, 125:175])
    bce = binary_crossentropy(y_true[:, 200:-1], y_pred[:, 200:-1])
    bce2 = binary_crossentropy(y_true[:, -1], y_pred[:, -1])
    return 2 * ((l1+l2+l3+l4+l5+l6)/6) + 1 * bce2+2*(bce*y_true[:, -1])

def custom_loss_bce_simple(y_true, y_pred):
    l1 = binary_crossentropy(y_true[:, :200], y_pred[:, :200])
    bce = binary_crossentropy(y_true[:,200 :-1], y_pred[:,200 :-1])
    bce2 = binary_crossentropy(y_true[:, -1], y_pred[:, -1])
    return 2* l1 + 1 * bce2+2*(bce*y_true[:, -1])


def custom_loss_combined(y_true, y_pred):
    # bce = binary_crossentropy(y_true[:, :-1], y_pred[:, :-1])
    l1 = binary_crossentropy(y_true[:, :25], y_pred[:, :25])
    l2 = binary_crossentropy(y_true[:, 75:100], y_pred[:, 75:100])
    l3 = binary_crossentropy(y_true[:, 100:125], y_pred[:, 100:125])
    l4 = binary_crossentropy(y_true[:, 175:200], y_pred[:, 175:200])
    l5 = binary_crossentropy(y_true[:, 25:75], y_pred[:, 25:75])
    l6 = binary_crossentropy(y_true[:, 125:175], y_pred[:, 125:175])


    l11 = mean_absolute_error(y_true[:, :25], y_pred[:, :25])
    l22 = mean_absolute_error(y_true[:,75 :100], y_pred[:,75 :100])
    l33 = mean_absolute_error(y_true[:, 100:125], y_pred[:, 100:125])
    l44 = mean_absolute_error(y_true[:, 175:200], y_pred[:, 175:200])
    l55 = mean_absolute_error(y_true[:, 25:75], y_pred[:, 25:75])
    l66 = mean_absolute_error(y_true[:, 125:175], y_pred[:, 125:175])

    bce = binary_crossentropy(y_true[:,200 :-1], y_pred[:,200 :-1])
    bce2 = binary_crossentropy(y_true[:, -1], y_pred[:, -1])
    return 1 * ((l1+l2+l3+l4+l5+l6)/6) +3 * ((l11+l22+l33+l44+l55+l66)/6) + 1 * bce2+2*(bce*y_true[:, -1])


def custom_loss_mse(y_true, y_pred):
    mse = mean_squared_error(y_true[:, :-1], y_pred[:, :-1])
    bce2 = binary_crossentropy(y_true[:, -1], y_pred[:, -1])
    return 0.99 * mse + 0.1 * bce2


def custom_loss_mae(y_true, y_pred):
    mae = mean_absolute_error(y_true[:, :-1], y_pred[:, :-1])
    bce2 = binary_crossentropy(y_true[:, -1], y_pred[:, -1])
    return 0.99 * mae + 0.1 * bce2

def custom_loss_css(y_true, y_pred):
    css1 = cosine_similarity(y_true[:, :25], y_pred[:, :25])
    css2 = cosine_similarity(y_true[:,75 :100], y_pred[:,75 :100])
    css3 = cosine_similarity(y_true[:, 100:125], y_pred[:, 100:125])
    css4 = cosine_similarity(y_true[:, 175:200], y_pred[:, 175:200])
    css5 = cosine_similarity(y_true[:, 25:75], y_pred[:, 25:75])
    css6 = cosine_similarity(y_true[:, 125:175], y_pred[:, 125:175])
    bce2 = binary_crossentropy(y_true[:, -1], y_pred[:, -1])
    return 0.99 * ((css1+css2+css3+css4+css5+css6)/6) + 0.1 * bce2


def custom_loss_kld(y_true, y_pred):
    kld = kullback_leibler_divergence(y_true[:, :-1], y_pred[:, :-1])
    # kld = tf.reduce_sum(kld , axis=0)
    # kld = tf.reduce_mean(kld )
    bce2 = binary_crossentropy(y_true[:, -1], y_pred[:, -1])
    return 0.00099 * kld + 0.1 * bce2





def make_model(network='vgg16',
               weights='imagenet',
               loss='bce',
               activation='sigmoid',
               optimizer='Adam',
               lr=0.001,
               width=100,
               height=100):
    if network == 'resnet50':
        Net = tf.keras.applications.ResNet50
    else:
        Net = tf.keras.applications.VGG16

    if loss == 'mse':
        loss = custom_loss_mse
    elif loss == 'mae':
        loss = custom_loss_mae
    elif loss == 'kld':
        loss = custom_loss_kld
    elif loss =='css':
        loss = cosine_similarity
    elif loss=='comb':
        loss = custom_loss_combined
    elif loss == 'bce_simp':
        loss = custom_loss_bce_simple
    else:
        loss = custom_loss_bce

    if optimizer == 'SGD':
        optimizer = SGD(learning_rate=lr, momentum=0.9)
    else:
        optimizer = Adam(lr=lr)


    net = Net(
        input_shape=[height, width, 3],
        include_top=False,
        weights=weights)
    for l in net.layers:
        l.trainable = True
    x = Flatten()(net.output)
    # x = Dense(1000, activation='relu')(x)
    # x = Dropout(.6)(x)
    # x = Dense(500, activation='relu')(x)
    # x = Dropout(.6)(x)
    x = Dense(width*2+5,
              kernel_initializer=GlorotNormal(),
              bias_initializer=Zeros(),
              activation=activation)(x)
    # x = tf.keras.activations.relu(x, alpha=0.0, max_value=1, threshold=0)
    model = Model(net.input, x)
    #   model.compile(loss=custom_loss, optimizer=SGD(learning_rate=0.01, momentum=0.9))
    model.compile(loss=loss,
                  run_eagerly=True,
                  optimizer=optimizer,
                  metrics=[tf.keras.metrics.MeanSquaredError(
                      name="mse", dtype=None),
                      tf.keras.metrics.RootMeanSquaredError(
                          name="rmse", dtype=None
                      ),
                      tf.keras.metrics.BinaryCrossentropy(
                          name="bce", dtype=None
                      ),
                      tf.keras.metrics.MeanAbsoluteError(
                          name="mae", dtype=None),
                      # tf.keras.metrics.MeanAbsolutePercentageError(
                      #     name="mape", dtype=None),
                      tf.keras.metrics.MeanSquaredLogarithmicError(
                          name="msle", dtype=None
                      ),
                      tf.keras.metrics.CosineSimilarity(
                          name="css", dtype=None, axis=-1
                      )
                  ])
    return model


def read_img(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img


# def read_coords(coords, height):
#     coords = np.array(json.loads(coords))
#     coords[100:] = np.where((coords[:100] >= height - 1) | (coords[:100] <= 1), 0, coords[100:])
#     coords[:100] = np.where((coords[:100] >= height - 1) | (coords[:100] <= 1), height / 2, coords[:100])
#     return coords / height

def read_coords(coords, height, width):
    coords = np.array(json.loads(coords))
    coords[width:] = np.where((coords[:width] >= height - 1) | (coords[:width] <= 1), 0, coords[width:])
    coords[:width] = np.where((coords[:width] >= height - 1) | (coords[:width] <= 1), height / 2, coords[:width])

    xs = np.where(coords[width:-1] > 0)
    if xs[0].shape[0] > 0:
        ymin = coords[:width].min() / height
        ymax = (coords[:width] + coords[width:]).max() / height
        xmin = xs[0].min() / width
        xmax = xs[0].max() / width
        box = np.array((ymin, xmin, ymax - ymin, xmax - xmin))
    else:
        box = np.zeros((4))

    return np.append(coords / height, box)


def img_generator(df, height, width, batch_size=64, network='vgg16'):
    # generate image and targets
    if network == 'resnet50':
        preprocess_input = preprocess_input_resnet50
    else:
        preprocess_input = preprocess_input_vgg16
    while True:
        # Each epoch will have 50 batches. Why? No reason
        for _ in range(50):
            # ids = [np.random.randint(0, len(df)) for i in range(batch_size)]
            # print(ids)
            df = df.reset_index(drop = True)
            sample = df.sample(n=batch_size).copy()
            paths = sample['path'].tolist()

            X = [read_img(path) for path in paths]
            X = np.stack(X)
            X = preprocess_input(X)
            labels = sample['class'].tolist()
            coords = sample['coords'].tolist()
            Y = [read_coords(coord, height, width) for coord in coords]
            Y = np.stack(Y)
            Y = np.hstack([Y, np.array(labels).reshape(-1, 1)])

            yield X, Y


# def read_coords(path):
#     with rasterio.open(path) as src:
#         meta = src.meta
#
#     line = gpd.GeoDataFrame.from_file(path.replace('images/', 'lines/mask_').replace('tif', 'geojson'))
#     coords = line.iloc[0]['geometry'].coords.xy
#     rows, cols = rasterio.transform.rowcol(meta['transform'] * rasterio.Affine.translation(0, 100),
#                                            coords[0],
#                                            coords[1])
#     rows = np.array(rows)
#     rows = np.where(rows >= 99, 0, rows)
#     return rows / 100


# def img_generator(paths, batch_size=64):
#     # generate image and targets
#     while True:
#         # Each epoch will have 50 batches. Why? No reason
#         for _ in range(50):
#             ids = [np.random.randint(0, paths.shape[0]) for i in range(batch_size)]
#
#             paths_list = paths[ids].copy().tolist()
#
#             X = [read_img(path) for path in paths_list]
#             Y = np.array([read_coords(path) for path in paths_list])
#             X = np.stack(X)
#
#             labels = np.where(Y[:, 49] == 0, 0, 1)
#             Y = np.stack(Y)
#             Y = np.hstack([Y, np.array(labels).reshape(-1, 1)])
#
#             yield X, Y

def main():
    data_path = '/media/ml/ex4/seetree/seetree_data/polygons_dataset/tree_in_center_w100_h300'
    train = pd.read_csv(os.path.join(data_path, 'train.csv'))
    val = pd.read_csv(os.path.join(data_path, 'val.csv'))

    models_dir = 'models'

    params = {
        'ex': '205-bce',
        'net': 'resnet50',
        'lr': 0.0001,
        'wei': 'imagenet',
        'loss': 'bce_simp',
        'act': 'sigmoid',
        'opt': 'Adam',
        'w': 100,
        'h': 300,
        'bs': 32

    }
    s = '{}'
    for i in range(len(params.keys()) - 1):
        s += '-{}'
    logs_dir = 'logs' + '/' + s.format(*[i + '-' + str(j) for i, j in zip(params.keys(), params.values())])
    csv_dir = 'csvs' + '/' + s.format(*[i + '-' + str(j) for i, j in zip(params.keys(), params.values())])
    os.makedirs(logs_dir, exist_ok=False)
    os.makedirs(csv_dir, exist_ok=False)
    best_model_file = \
        '{}/'.format(models_dir) + 'epoch-{epoch:d}-' + s.format(
            *[i + '-' + str(j) for i, j in zip(params.keys(), params.values())]) + \
        '-loss-{val_loss:0.7f}-mse-{val_mse:0.7f}-rmse-{val_rmse:0.7f}-bce-{val_bce:0.7f}' + \
        '-mae-{val_mae:0.7f}-msle-{val_msle:0.7f}-css-{val_css:0.7f}' + \
        '.h5'

    model = make_model(params['net'],
                       params['wei'],
                       params['loss'],
                       params['act'],
                       params['opt'],
                       params['lr'],
                       params['w'],
                       params['h'])
    # model.load_weights(
    #     '/media/ml/ex4/seetree/seetree_dev/scratch/polygon_prediction/models/resnet50-22-loss-0.6111830-mse-0.0479484-rmse-0.2189712-mae-0.1051554-mape-80671456.0000000-msle-0.0274004-css-0.6719598.h5')

    best_model = ModelCheckpoint(best_model_file, monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='min')
    csv_logger = CSVLogger(os.path.join(csv_dir, 'training.log'))

    tensorboard = TensorBoard(log_dir=logs_dir, histogram_freq=1, write_graph=True, write_images=True, update_freq=10)

    callbacks = [best_model,
                 csv_logger,
                 tensorboard
                 ]

    r = model.fit(
        img_generator(train,
                      params['h'],
                      params['w'],
                      params['bs'],
                      params['net']),
        validation_data=img_generator(val,
                                      params['h'],
                                      params['w'],
                                      params['bs'],
                                      params['net']),
        steps_per_epoch=25,
        validation_steps=12,
        epochs=300,
        callbacks=callbacks
    )

    print(r.history.keys())

    # plot some data
    plt.plot(r.history['loss'], label='loss')
    plt.plot(r.history['val_loss'], label='val_loss')
    plt.legend()
    plt.show()

    # accuracies
    plt.plot(r.history['mse'], label='acc')
    plt.plot(r.history['val_mse'], label='val_acc')
    plt.legend()
    plt.show()


if __name__ == '__main__':
    main()

# New Section